## Preprocessing data to prepare for Machine Learning with stock data

https://pythonprogramming.net/preprocessing-for-machine-learning-python-programming-for-finance/

Hello and welcome to part 9 of the Python for Finance tutorial series. In the previous tutorials, we've covered how to pull in stock pricing data for a large number of companies, how to combine that data into one large dataset, and how to visually represent at least one relationship between all of the companies. Now, we're going to try to take this data and do some machine learning with it!

The idea is to see what might happen if we took data from all of the current companies, and fed this through some sort of machine learning classifier. We know that, over time, various companies have different relationships with eachother, so, if the machine can recognize and fit these relationships, it's possible we could predict from changes in prices today, what will happen tomorrow with a specific company. Let's try!

To begin, all machine learning does is take "featuresets" and attempts to map them to "labels." Whether we're doing [K Nearest Neighbors](https://pythonprogramming.net/k-nearest-neighbors-intro-machine-learning-tutorial/) or [deep learning with neural networks](https://pythonprogramming.net/neural-networks-machine-learning-tutorial/), this remains the same. Thus, we need to convert our existing data to featuresets and labels.

Our features can be other company's prices, but we're going to instead say the features are the pricing changes that day for all companies. Our label will be whether or not we actually want to buy a specific company. Let's say we're considering Exxon (XOM). What we'll do for featuresets is take into account all company percent changes that day, and those will be our features. Our label will be whether or not Exxon (XOM) rose more than x% within the next x days, where we can pick whatever we want for x. To start, let's say a company is a buy if, within the next 7 days, its price goes up more than 2% and it is a sell if the price goes down more than 2% within those 7 days.

This is something we could also relatively easily make a strategy for. If the algorithm says buy, we can buy, place a 2% drop stop-loss (basically something that tells the exchange is price falls below this number / or goes above if you're shorting the company, then exit my position). Otherwise, sell the company once it has risen 2%, or you could be conservative and sell at 1% rise...etc. Regardless, you could relatively easily build a strategy from this classifier. In order to begin, we need the prices into the future for our training data.

I am going to keep coding in our same script. If this is a problem to you, feel free to create a new file and import the functions we use.

Full code up to this point:

In [1]:
import bs4 as bs
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import os
import pandas as pd
import yfinance as yf
import pickle
import requests
from tqdm import tqdm
from IPython.display import display


def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.strip())
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers


def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # e.g. BRK.B => BRK-B, BF.B => BF-B
        ticker = ticker.replace('.', '-')
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = yf.download(ticker, start=start, end=end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for ticker in tqdm(tickers):
        # e.g. BRK.B => BRK-B, BF.B => BF-B
        ticker = ticker.replace('.', '-')
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

    display(main_df)
    main_df.to_csv('sp500_joined_closes.csv')


def visualize_data():
    df = pd.read_csv('sp500_joined_closes.csv')
    df_corr = df.corr(numeric_only=True)
    display(df_corr.head())
    df_corr.to_csv('sp500corr.csv')
    data1 = df_corr.values
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)

    heatmap1 = ax1.pcolor(data1, cmap=plt.cm.RdYlGn)
    fig1.colorbar(heatmap1)

    ax1.set_xticks(np.arange(data1.shape[1]) + 0.5, minor=False)
    ax1.set_yticks(np.arange(data1.shape[0]) + 0.5, minor=False)
    ax1.invert_yaxis()
    ax1.xaxis.tick_top()
    column_labels = df_corr.columns
    row_labels = df_corr.index
    ax1.set_xticklabels(column_labels)
    ax1.set_yticklabels(row_labels)
    plt.xticks(rotation=90)
    heatmap1.set_clim(-1, 1)
    plt.tight_layout()
    plt.show()

    
# visualize_data()

Continuing along, let's begin to process some data that will help us to create our labels:

In [2]:
# def process_data_for_labels(ticker):
#     hm_days = 7
#     df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
#     tickers = df.columns.values.tolist()
#     df.fillna(0, inplace=True)

This function will take one parameter: the `ticker` in question. Each model will be trained on a single company. Next, we want to know how many days into the future we need prices for. We're choosing 7 here. Now, we'll read in the data for the close prices for all companies that we've saved in the past, grab a list of the existing tickers, and we'll fill any missing with 0 for now. This might be something you want to change in the future, but we'll go with 0 for now. Now, we want to grab the % change values for the next 7 days:

In [3]:
#     for i in range(1,hm_days+1):
#         df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

This creates new dataframe columns for our specific ticker in question, using [string formatting](https://pythonprogramming.net/string-concatenation-formatting-intermediate-python-tutorial/) to create the custom names. The way we're getting future values is with `.shift`, which basically will shift a column up or down. In this case, we shift a negative amount, which will take that column and, if you could see it visually, it would shift that column `UP` by `i` rows. This gives us the future values i days in advanced, which we can calculate percent change against.

Finally:

In [4]:
#     df.fillna(0, inplace=True)
#     return tickers, df

We're all set here, we'll return the tickers and the dataframe, and we're well on our way to having some featuresets that our algorithms can use to try to fit and find relationships.

Our full processing function:

In [5]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df

In [6]:
tickers, df = process_data_for_labels('XOM')

In [7]:
df[['XOM', 'XOM_1d', 'XOM_2d', 'XOM_3d', 'XOM_4d', 'XOM_5d', 'XOM_6d', 'XOM_7d']]

,XOM,XOM_1d,XOM_2d,XOM_3d,XOM_4d,XOM_5d,XOM_6d,XOM_7d
Date,,,,,,,,
2010-01-04,40.983105,0.003905,0.012581,0.009400,0.005350,0.016631,0.011569,0.007520
2010-01-05,41.143135,0.008643,0.005474,0.001440,0.012676,0.007635,0.003601,0.003745
2010-01-06,41.498730,-0.003142,-0.007141,0.003999,-0.001000,-0.004999,-0.004856,-0.012996
2010-01-07,41.368336,-0.004012,0.007164,0.002149,-0.001862,-0.001719,-0.009885,-0.007593
2010-01-08,41.202381,0.011220,0.006186,0.002158,0.002302,-0.005897,-0.003596,-0.021433
...,...,...,...,...,...,...,...,...
2023-10-24,108.389999,0.001845,-0.007289,-0.026202,-0.023157,0.000000,0.000000,0.000000
2023-10-25,108.589996,-0.009117,-0.027995,-0.024956,0.000000,0.000000,0.000000,0.000000
2023-10-26,107.599998,-0.019052,-0.015985,0.000000,0.000000,0.000000,0.000000,0.000000


In the next tutorial, we're going to cover how we'll go about creating our "labels."